In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
import pandas as pd
import numpy as np

# Step 1: Load CSV Data
df = pd.read_csv('/Users/Nufail/Desktop/Process_Mining/Process_Mining/conformance_checking/data/df_for_decision_tree.csv')

In [3]:
df.head()

trace_number  start_log_moves  start_model_moves  \
0             0                0                  0   
1             1                0                  0   
2             2                0                  0   
3             3                0                  0   
4             4                0                  0   

   A-Create-Application_log_moves  A-Create-Application_model_moves  \
0                               0                                 0   
1                               0                                 0   
2                               0                                 0   
3                               0                                 0   
4                               0                                 0   

   A-Submitted_log_moves  A-Submitted_model_moves  \
0                    0.0                      0.0   
1                    0.0                      0.0   
2                    NaN                      NaN   
3                    NaN                      NaN   
4                    0.0                      0.0   

   W-Handle-leads-schedule_log_moves  W-Handle-leads-schedule_model_moves  \
0                                0.0                                  0.0   
1                                0.0                                  0.0   
2                                NaN                                  NaN   
3                                NaN                                  NaN   
4                                0.0                                  0.0   

   W-Handle-leads-withdraw_log_moves  ...  \
0                                0.0  ...   
1                                0.0  ...   
2                                NaN  ...   
3                                NaN  ...   
4                                0.0  ...   

   W-Handle-leads-ate-abort_model_moves  \
0                                   NaN   
1                                   NaN   
2                                   NaN   
3                                   NaN   
4                                   NaN   

   W-Shortened-completion-schedule_log_moves  \
0                                        NaN   
1                                        NaN   
2                                        NaN   
3                                        NaN   
4                                        NaN   

   W-Shortened-completion-schedule_model_moves  \
0                                          NaN   
1                                          NaN   
2                                          NaN   
3                                          NaN   
4                                          NaN   

   W-Shortened-completion-start_log_moves  \
0                                     NaN   
1                                     NaN   
2                                     NaN   
3                                     NaN   
4                                     NaN   

   W-Shortened-completion-start_model_moves  \
0                                       NaN   
1                                       NaN   
2                                       NaN   
3                                       NaN   
4                                       NaN   

   W-Shortened-completion-suspend_log_moves  \
0                                       NaN   
1                                       NaN   
2                                       NaN   
3                                       NaN   
4                                       NaN   

   W-Shortened-completion-suspend_model_moves  \
0                                         NaN   
1                                         NaN   
2                                         NaN   
3                                         NaN   
4                                         NaN   

   W-Call-after-offers-withdraw_log_moves  \
0                                     NaN   
1                                     NaN   
2                                     NaN   
3                              

In [25]:
df.shape

(3093, 112)

In [9]:
type(df['case:throughput_time'])

pandas.core.series.Series

# Set Up Decision tree param

In [12]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn import tree
import matplotlib.pyplot as plt
import json



# Fill NaN values with -1 to indicate actions that did not appear in the trace
df_filled = df.fillna(-1)

# Exclude unnecessary columns (trace number and "None" moves)
columns_to_exclude = ["trace_number", "None_log_moves", "None_model_moves"]
X = df_filled.drop(columns=columns_to_exclude + ["case:throughput_time"])
y = df_filled["case:throughput_time"]  # We'll convert this to hours later

# Step 3: Convert 'case:throughput_time' to numeric (hours) for modeling
# Ensure 'case:throughput_time' is in timedelta format
y_timedelta = pd.to_timedelta(y)
y_hours = y_timedelta.apply(lambda x: x.total_seconds() / 3600)  # Convert timedelta to hours

# Step 4: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_hours, test_size=0.2, random_state=42)

# Step 5: Define a function to load parameter grid from a JSON file
def load_param_grid(json_file_path: str) -> dict:
    """
    Loads the parameter grid for GridSearchCV from a JSON file.

    Parameters
    ----------
    json_file_path : str
        The path to the JSON file containing the parameter grid.

    Returns
    -------
    dict
        The parameter grid dictionary.
    """
    with open(json_file_path, 'r') as file:
        param_grid = json.load(file)
    return param_grid

# Load the parameter grid from a JSON file
param_grid_path = 'param_grid.json'  # Replace with the actual path to your JSON file
param_grid = load_param_grid(param_grid_path)

# Define the Decision Tree model with restricted hyperparameters for interpretability
tree_regressor = DecisionTreeRegressor(random_state=42)

# Step 7: Perform Grid Search with Cross-Validation using the loaded param_grid
grid_search = GridSearchCV(
    estimator=tree_regressor,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=2
)
grid_search.fit(X_train, y_train)

print("Best Parameters found in Grid Search:", grid_search.best_params_)


Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=60; total time=   0.0s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=60; total time=   0.0s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=60; total time=   0.0s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=60; total time=   0.0s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=60; total time=   0.1s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=80; total time=   0.0s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=80; total time=   0.0s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=80; total time=   0.1s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=100; total time=   0.0s
[CV] END max_depth=3, max_features=5,

# See best tree

In [13]:

# Extract the best model from the grid search
best_tree_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred_hours = best_tree_model.predict(X_test)

# Convert predictions back to timedelta format for interpretation
y_pred_timedelta = pd.Series(y_pred_hours).apply(lambda hours: pd.to_timedelta(hours, unit='h'))
y_test_timedelta = pd.Series(y_test).apply(lambda hours: pd.to_timedelta(hours, unit='h'))

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred_hours)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_hours))
r2 = r2_score(y_test, y_pred_hours)

print("Model Performance:")
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Mean Absolute Error (MAE): {mae:.2f} hours")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f} hours")
print(f"R-squared (R2): {r2:.2f}")

# Display first few results for interpretation
results_df = pd.DataFrame({
    'Actual Throughput Time (Hours)': y_test_timedelta.reset_index(drop=True),
    'Predicted Throughput Time (Hours)': y_pred_timedelta
})
print(results_df.head())

# Visualize the decision tree to understand decision rules
plt.figure(figsize=(15, 10))
tree.plot_tree(best_tree_model, feature_names=X.columns, filled=True, rounded=True, fontsize=8)
plt.title("Decision Tree Visualization")
plt.show()

Model Performance:
Best Parameters: {'max_depth': 5, 'max_features': 12, 'min_samples_leaf': 50, 'min_samples_split': 60}
Mean Absolute Error (MAE): 163.14 hours
Root Mean Squared Error (RMSE): 234.49 hours
R-squared (R2): 0.17
  Actual Throughput Time (Hours) Predicted Throughput Time (Hours)
0     20 days 00:56:41.258000400        15 days 14:01:44.561809200
1     41 days 08:53:06.929998800        16 days 06:42:27.698896800
2      8 days 11:57:34.908998400        17 days 18:20:31.059146400
3     21 days 18:28:18.203998800        17 days 09:39:03.999135600
4      0 days 00:57:59.281999200         6 days 19:27:58.216856400


# Get Rules

In [17]:
from typing import List, Dict, Tuple
import pandas as pd
from sklearn.tree import _tree

def extract_rules_from_tree(decision_tree_model, feature_names: List[str], target_name: str) -> List[Dict[str, any]]:
    """
    Extract rules by traversing from the root to each leaf in the trained decision tree model.
    
    Parameters
    ----------
    decision_tree_model : sklearn.tree.DecisionTreeRegressor
        The trained decision tree model from which to extract path-based rules.
    feature_names : List[str]
        List of feature names to interpret tree splits.
    target_name : str
        The name of the target variable, for leaf-level output interpretation.
        
    Returns
    -------
    List[Dict[str, any]]
        A list of dictionaries, each representing a rule with:
            - 'conditions': List of conditions (feature, operator, threshold) for the path.
            - 'average_gini': Average Gini impurity encountered along the path.
            - 'final_gini': Gini impurity at the leaf node.
            - 'average_throughput_time': Average value of target (throughput time) at the leaf node.
    """
    tree_ = decision_tree_model.tree_
    rules = []

    def traverse(node: int, conditions: List[Tuple[str, str, float]], gini_values: List[float]):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:  # Not a leaf node
            # Extract feature name and threshold
            feature = feature_names[tree_.feature[node]]
            threshold = tree_.threshold[node]
            gini = tree_.impurity[node]

            # Traverse left (feature <= threshold)
            left_conditions = conditions + [(feature, "<=", threshold)]
            traverse(tree_.children_left[node], left_conditions, gini_values + [gini])

            # Traverse right (feature > threshold)
            right_conditions = conditions + [(feature, ">", threshold)]
            traverse(tree_.children_right[node], right_conditions, gini_values + [gini])
        
        else:
            # Leaf node, create a rule
            final_gini = tree_.impurity[node]
            avg_gini = sum(gini_values) / len(gini_values) if gini_values else final_gini
            avg_throughput_time = tree_.value[node][0][0]  # Mean target value at leaf
            
            rule = {
                'conditions': conditions,
                'average_gini': avg_gini,
                'final_gini': final_gini,
                'average_throughput_time': avg_throughput_time
            }
            rules.append(rule)

    # Start traversal from the root node
    traverse(0, [], [])
    return rules


In [29]:
def split_log_by_rules_with_labels(log_df: pd.DataFrame, rules: List[Dict[str, any]]) -> Dict[str, pd.DataFrame]:
    """
    Split the log into segments based on the full path rules (root to leaf) 
    and label each trace with the rule it matches. Include a separate segment for unmatched traces.

    Parameters
    ----------
    log_df : pd.DataFrame
        The original event log data.
    rules : List[Dict[str, any]]
        Extracted path-based rules from the decision tree, where each rule includes:
            - conditions: List of conditions (feature, operator, threshold).
            - average_gini: Average Gini impurity along the path.
            - final_gini: Gini impurity at the leaf.
            - average_throughput_time: Average throughput time at the leaf node.

    Returns
    -------
    Dict[str, pd.DataFrame]
        A dictionary containing:
            - Segments for traces matching each rule (labeled by rule index, starting from 1).
            - A segment for unmatched traces under the key "unmatched".
    """
    log_with_labels = log_df.copy()
    log_with_labels['rule_index'] = -1  # Default to -1 for unmatched traces

    segments = {}

    for rule_idx, rule in enumerate(rules, start=1):  # Start counting rules from 1
        rule_filter = pd.Series([True] * len(log_df))  # Start with all True

        for feature, op, threshold in rule['conditions']:
            if op == "<=":
                rule_filter &= (log_df[feature] <= threshold)
            elif op == ">":
                rule_filter &= (log_df[feature] > threshold)
            else:
                raise ValueError(f"Unsupported operator '{op}' in rule.")
        
        # Apply the rule filter and label matching traces
        matched_traces = log_with_labels[rule_filter].copy()
        if not matched_traces.empty:
            matched_traces['rule_index'] = rule_idx
            segments[f"Segment {rule_idx}"] = matched_traces
            print(f"Segment {rule_idx} created with {len(matched_traces)} traces (matches Rule {rule_idx}).")
        else:
            print(f"Rule {rule_idx} did not match any traces.")

        # Update the main DataFrame with rule labels
        log_with_labels.loc[rule_filter, 'rule_index'] = rule_idx

    # Handle unmatched traces
    unmatched_traces = log_with_labels[log_with_labels['rule_index'] == -1]
    if not unmatched_traces.empty:
        segments["unmatched"] = unmatched_traces
        print(f"Unmatched Segment created with {len(unmatched_traces)} traces.")

    return segments


In [18]:
# Extract rules from the decision tree model
feature_names = X.columns  # Features used for training
rules = extract_rules_from_tree(best_tree_model, feature_names, target_name="case:throughput_time")

# Print extracted rules for inspection
print("Extracted Rules:")
for idx, rule in enumerate(rules, 1):
    print(f"Rule {idx}:")
    print("Conditions:")
    for feature, operator, threshold in rule['conditions']:
        print(f"  - {feature} {operator} {threshold}")
    print(f"  Average Gini Impurity Along Path: {rule['average_gini']:.4f}")
    print(f"  Final Gini Impurity at Leaf: {rule['final_gini']:.4f}")
    print(f"  Average Throughput Time (Hours): {rule['average_throughput_time']:.2f}\n")


Extracted Rules:
Rule 1:
Conditions:
  - W-Call-incomplete-files-start_model_moves <= -0.5
  - W-Validate-application-suspend_model_moves <= -0.5
  Average Gini Impurity Along Path: 67226.9067
  Final Gini Impurity at Leaf: 35430.5768
  Average Throughput Time (Hours): 163.47

Rule 2:
Conditions:
  - W-Call-incomplete-files-start_model_moves <= -0.5
  - W-Validate-application-suspend_model_moves > -0.5
  - W-Call-after-offers-suspend_log_moves <= 0.5
  - W-Validate-application-ate-abort_log_moves <= -0.5
  Average Gini Impurity Along Path: 54469.2809
  Final Gini Impurity at Leaf: 15132.3800
  Average Throughput Time (Hours): 229.82

Rule 3:
Conditions:
  - W-Call-incomplete-files-start_model_moves <= -0.5
  - W-Validate-application-suspend_model_moves > -0.5
  - W-Call-after-offers-suspend_log_moves <= 0.5
  - W-Validate-application-ate-abort_log_moves > -0.5
  Average Gini Impurity Along Path: 54469.2809
  Final Gini Impurity at Leaf: 25781.8912
  Average Throughput Time (Hours): 241

## Segment Event log

In [30]:
# Segment the log based on the extracted rules
segmented_logs = split_log_by_rules_with_labels(df_filled, rules)


Segment 1 created with 139 traces (matches Rule 1).
Segment 2 created with 96 traces (matches Rule 2).
Segment 3 created with 74 traces (matches Rule 3).
Segment 4 created with 790 traces (matches Rule 4).
Segment 5 created with 866 traces (matches Rule 5).
Segment 6 created with 77 traces (matches Rule 6).
Segment 7 created with 75 traces (matches Rule 7).
Segment 8 created with 135 traces (matches Rule 8).
Segment 9 created with 227 traces (matches Rule 9).
Segment 10 created with 190 traces (matches Rule 10).
Segment 11 created with 256 traces (matches Rule 11).
Segment 12 created with 168 traces (matches Rule 12).


# Synthetic Event Logs

In [183]:
from pm4py.objects.petri_net.importer import importer as pnml_importer

petri_net, initial_marking, final_marking = pnml_importer.apply('/Users/Nufail/Desktop/Process_Mining/Process_Mining/conformance_checking/data/BPI2017Denied_petriNet.pnml')

In [227]:
import pm4py

# Generate synthetic event log from the Petri net
synth_log = pm4py.sim.play_out(petri_net, initial_marking, final_marking)

# Convert synthetic log to Pandas DataFrame
log_data = []
for trace in synth_log:
    trace_id = trace.attributes['concept:name']  # Access attributes as object properties
    for event in trace:
        log_data.append({
            'trace_id': trace_id,
            'activity': event['concept:name'],  # Event attributes are accessed this way
            'timestamp': event['time:timestamp']
        })

# Create a DataFrame for the synthetic event log
synth_log_df = pd.DataFrame(log_data)


In [228]:
ldld = synth_log_df[synth_log_df['trace_id'] == '1']
ldld

trace_id                           activity                 timestamp
29        1                              start 1970-04-26 18:47:09+00:00
30        1               A-Create-Application 1970-04-26 18:47:10+00:00
31        1                        A-Submitted 1970-04-26 18:47:11+00:00
32        1            W-Handle-leads-schedule 1970-04-26 18:47:12+00:00
33        1            W-Handle-leads-withdraw 1970-04-26 18:47:13+00:00
34        1  W-Assess-potential-fraud-schedule 1970-04-26 18:47:14+00:00
35        1     W-Assess-potential-fraud-start 1970-04-26 18:47:15+00:00
36        1   W-Assess-potential-fraud-suspend 1970-04-26 18:47:16+00:00
37        1    W-Assess-potential-fraud-resume 1970-04-26 18:47:17+00:00
38        1   W-Assess-potential-fraud-suspend 1970-04-26 18:47:18+00:00
39        1                                end 1970-04-26 18:47:19+00:00

In [229]:
import pandas as pd
import random
from datetime import timedelta

def adjust_timestamps(df, trace_column='trace_id', timestamp_column='timestamp', min_increment=2400, max_increment=6000):
    """
    Adjust timestamps within each trace to add realistic time differences while maintaining chronological order.
    
    Parameters
    ----------
    df : pd.DataFrame
        The event log DataFrame containing traces with activities and timestamps.
    trace_column : str
        The column name representing trace identifiers.
    timestamp_column : str
        The column name representing activity timestamps.
    min_increment : int
        Minimum increment (in seconds) between consecutive activities.
    max_increment : int
        Maximum increment (in seconds) between consecutive activities.
        
    Returns
    -------
    pd.DataFrame
        The updated DataFrame with adjusted timestamps.
    """
    # Copy the DataFrame to avoid modifying the original data
    df = df.copy()
    
    # Convert timestamps to pandas datetime if not already
    df[timestamp_column] = pd.to_datetime(df[timestamp_column], utc=True)
    
    # Group by trace and process each trace
    for trace_id, group in df.groupby(trace_column):
        # Sort the group by timestamp to ensure order
        group = group.sort_values(by=timestamp_column)
        adjusted_timestamps = []
        current_time = group.iloc[0][timestamp_column]  # Start with the first timestamp
        
        # Generate adjusted timestamps for the trace
        for _ in range(len(group)):
            adjusted_timestamps.append(current_time)
            # Increment the current time by a random value within the range
            increment = timedelta(seconds=random.randint(min_increment, max_increment))
            current_time += increment
        
        # Update the original DataFrame with adjusted timestamps
        df.loc[group.index, timestamp_column] = adjusted_timestamps

    return df

# Example of how to use this function (replace 'synth_log_df' with your actual DataFrame variable name)
# Updated timestamps
synth_log_df_adjusted = adjust_timestamps(synth_log_df, trace_column='trace_id', timestamp_column='timestamp')



In [230]:
synth_log_df_adjusted[synth_log_df_adjusted['trace_id'] == '1']


trace_id                           activity                 timestamp
29        1                              start 1970-04-26 18:47:09+00:00
30        1               A-Create-Application 1970-04-26 19:33:04+00:00
31        1                        A-Submitted 1970-04-26 20:34:56+00:00
32        1            W-Handle-leads-schedule 1970-04-26 22:08:44+00:00
33        1            W-Handle-leads-withdraw 1970-04-26 22:52:33+00:00
34        1  W-Assess-potential-fraud-schedule 1970-04-27 00:31:54+00:00
35        1     W-Assess-potential-fraud-start 1970-04-27 01:31:44+00:00
36        1   W-Assess-potential-fraud-suspend 1970-04-27 03:09:56+00:00
37        1    W-Assess-potential-fraud-resume 1970-04-27 04:18:44+00:00
38        1   W-Assess-potential-fraud-suspend 1970-04-27 05:14:34+00:00
39        1                                end 1970-04-27 06:28:50+00:00

In [231]:
# Group and count unique activities
activity_counts = synth_log_df['activity'].value_counts().reset_index()

# Rename columns for clarity
activity_counts.columns = ['activity', 'count']

# Display the resulting dataframe
activity_counts


activity  count
0     W-Call-incomplete-files-suspend   1938
1      W-Complete-application-suspend   1087
2   W-Call-incomplete-files-ate-abort   1015
3                        A-Incomplete   1015
4       W-Call-incomplete-files-start   1015
5    W-Call-incomplete-files-schedule   1015
6                               start   1000
7                                 end   1000
8                A-Create-Application   1000
9    W-Assess-potential-fraud-suspend    934
10     W-Call-incomplete-files-resume    923
11      W-Complete-application-resume    545
12                     O-Create-Offer    525
13        W-Call-after-offers-suspend    525
14         W-Call-after-offers-resume    525
15      W-Call-after-offers-ate-abort    525
16          W-Call-after-offers-start    525
17       W-Call-after-offers-schedule    525
18                          O-Created    525
19             O-Sent-mail-and-online    525
20                         A-Complete    525
21                         A-Accepted    525
22                          A-Concept    525
23       W-Complete-application-start    525
24    W-Complete-application-schedule    525
25              W-Handle-leads-resume    501
26             W-Handle-leads-suspend    501
27                        A-Submitted    483
28            W-Handle-leads-schedule    483
29     W-Assess-potential-fraud-start    475
30  W-Assess-potential-fraud-schedule    475
31    W-Assess-potential-fraud-resume    453
32                           A-Denied    358
33                          O-Refused    358
34    W-Validate-application-complete    340
35    W-Validate-application-schedule    336
36   W-Validate-application-ate-abort    332
37                         O-Returned    331
38       W-Validate-application-start    327
39                       A-Validating    327
40     W-Validate-application-suspend    319
41    W-Shortened-completion-schedule    272
42       W-Shortened-completion-start    272
43                        O-Cancelled    271
44    W-Complete-application-complete    263
45   W-Complete-application-ate-abort    262
46            W-Handle-leads-withdraw    243
47               W-Handle-leads-start    240
48       W-Call-after-offers-complete    192
49      W-Validate-application-resume    163
50       W-Call-after-offers-withdraw    154
51           W-Handle-leads-ate-abort    121
52     W-Shortened-completion-suspend    121
53            W-Handle-leads-complete    119

In [232]:
import pandas as pd
import numpy as np

def introduce_synthetic_deviations_W(
    log_df: pd.DataFrame,
    activities_to_skip: list,
    deviation_ratio: float = 0.3,
    time_reduction_range: tuple = (3600, 10800),  # Time reduction range in seconds
    seed: int = 42
) -> pd.DataFrame:
    """
    Introduces synthetic deviations by skipping user-specified activities in a subset of traces.

    Parameters
    ----------
    log_df : pd.DataFrame
        The synthetic event log as a DataFrame with columns ['trace_id', 'activity', 'timestamp'].
    activities_to_skip : list
        List of activities to skip (remove) from traces.
    deviation_ratio : float, optional
        The ratio of traces containing the specified activity to modify (default is 0.3, i.e., 30%).
    time_reduction_range : tuple, optional
        Range of time (in seconds) to reduce for subsequent events (default is (60, 3600)).
    seed : int, optional
        Random seed for reproducibility (default is 42).

    Returns
    -------
    pd.DataFrame
        A modified event log with synthetic deviations introduced.
    """
    np.random.seed(seed)
    modified_log = log_df.copy()

    for activity in activities_to_skip:
        # Identify traces containing the activity
        traces_with_activity = modified_log[modified_log['activity'] == activity]['trace_id'].unique()

        # Randomly select traces to modify based on the deviation ratio
        num_traces_to_modify = int(len(traces_with_activity) * deviation_ratio)
        traces_to_modify = np.random.choice(traces_with_activity, num_traces_to_modify, replace=False)

        for trace_id in traces_to_modify:
            # Filter the trace
            trace = modified_log[modified_log['trace_id'] == trace_id].copy()

            # Find indices of activities to be removed
            indices_to_remove = trace[trace['activity'] == activity].index

            # Determine the first affected index
            first_affected_index = indices_to_remove.min()

            # Remove all occurrences of the activity
            trace = trace.drop(indices_to_remove)

            # Generate a random time reduction in seconds
            time_reduction = pd.Timedelta(seconds=np.random.uniform(*time_reduction_range))

            # Adjust timestamps for activities after the last removed activity
            if not trace.empty and first_affected_index is not None:
                # Identify indices of events following the removed activities
                subsequent_indices = trace.index[trace.index > first_affected_index]

                # Adjust their timestamps
                trace.loc[subsequent_indices, 'timestamp'] -= time_reduction

            # Replace the trace in the modified log
            modified_log = modified_log[modified_log['trace_id'] != trace_id]
            modified_log = pd.concat([modified_log, trace])

    # Sort the modified log by trace_id and timestamp
    modified_log = modified_log.sort_values(by=['trace_id', 'timestamp']).reset_index(drop=True)
    return modified_log


In [190]:
# Example usage
synthetic_log_deviat = introduce_synthetic_deviations_W(
    log_df=synth_log_df_adjusted,
    activities_to_skip=['W-Call-incomplete-files-resume', 'W-Call-incomplete-files-suspend', 'W-Assess-potential-fraud-suspend'],
    deviation_ratio=1,
    time_reduction_range=(3600, 10800),
    seed=49
)


In [192]:
synthetic_log_deviat[synthetic_log_deviat['trace_id'] == '1']




trace_id                           activity  \
9         1                              start   
10        1               A-Create-Application   
11        1  W-Assess-potential-fraud-schedule   
12        1     W-Assess-potential-fraud-start   
13        1    W-Assess-potential-fraud-resume   
14        1    W-Assess-potential-fraud-resume   
15        1                                end   

                             timestamp  
9            1970-04-26 18:46:50+00:00  
10           1970-04-26 20:02:15+00:00  
11           1970-04-26 20:46:21+00:00  
12           1970-04-26 22:20:04+00:00  
13 1970-04-27 00:14:38.610702861+00:00  
14 1970-04-27 02:19:27.610702861+00:00  
15 1970-04-27 03:23:08.610702861+00:00

## creating csv file of synthetic data

In [237]:
# Example usage
synthetic_log_deviat_test = introduce_synthetic_deviations_W(
    log_df=synth_log_df_adjusted,
    activities_to_skip=['W-Call-incomplete-files-suspend', 'A-Incomplete', 'W-Assess-potential-fraud-suspend'],
    deviation_ratio=0.6,
    time_reduction_range=(3600, 9600),
    seed=12
)

In [238]:
synthetic_log_deviat_test = synthetic_log_deviat_test.rename(
    columns={
        'trace_id': 'case:concept:name',
        'activity': 'concept:name',
        'timestamp': 'time:timestamp'
    }
)

In [239]:
synthetic_log_deviat_test['time:timestamp'] = pd.to_datetime(synthetic_log_deviat_test['time:timestamp'])

In [240]:
from pm4py.objects.petri_net.importer import importer as pnml_importer

petri_net, initial_marking, final_marking = pnml_importer.apply('/Users/Nufail/Desktop/Process_Mining/Process_Mining/conformance_checking/data/BPI2017Denied_petriNet.pnml')

In [241]:
alignments = pm4py.algo.conformance.alignments.petri_net.algorithm.apply_log(log=synthetic_log_deviat_test,
                                                                             petri_net=petri_net,
                                                                             initial_marking=initial_marking,
                                                                             final_marking=final_marking)


aligning log, completed variants :: 100%|██████████| 636/636 [00:59<00:00, 10.72it/s]


In [242]:
synthetic_log_deviat_test[synthetic_log_deviat_test['case:concept:name'] == '1']


case:concept:name                       concept:name  \
25                 1                              start   
26                 1               A-Create-Application   
27                 1                        A-Submitted   
28                 1            W-Handle-leads-schedule   
29                 1            W-Handle-leads-withdraw   
30                 1  W-Assess-potential-fraud-schedule   
31                 1     W-Assess-potential-fraud-start   
32                 1    W-Assess-potential-fraud-resume   
33                 1                                end   

                        time:timestamp  
25           1970-04-26 18:47:09+00:00  
26           1970-04-26 19:33:04+00:00  
27           1970-04-26 20:34:56+00:00  
28           1970-04-26 22:08:44+00:00  
29           1970-04-26 22:52:33+00:00  
30           1970-04-27 00:31:54+00:00  
31           1970-04-27 01:31:44+00:00  
32 1970-04-27 02:12:59.987970194+00:00  
33 1970-04-27 04:23:05.987970194+00:00

In [243]:
def map_alignments_with_trace_ids(log_df, alignments):
    """
    Map alignments to their respective trace IDs from the event log.

    Parameters
    ----------
    log_df : pd.DataFrame
        Event log as a DataFrame containing 'case:concept:name'.
    alignments : List
        List of alignment dictionaries from the conformance checking.

    Returns
    -------
    List[Dict[str, Any]]
        A list where each element contains:
            - 'trace_id': The trace ID
            - 'alignment': The corresponding alignment
    """
    # Ensure trace IDs are unique and in order
    unique_trace_ids = log_df['case:concept:name'].unique()

    if len(unique_trace_ids) != len(alignments):
        raise ValueError("Mismatch between the number of traces in the log and alignments!")

    # Map alignments to trace IDs
    alignments_with_trace_ids = [
        {"trace_id": trace_id, "alignment": alignment}
        for trace_id, alignment in zip(unique_trace_ids, alignments)
    ]

    return alignments_with_trace_ids


In [244]:
# Map alignments to trace IDs
alignments_with_trace_ids = map_alignments_with_trace_ids(synthetic_log_deviat_test, alignments)


In [245]:
def generate_trace_encoding(alignments_with_trace_ids):
    """
    Generate trace encodings and associate them with trace IDs.

    Parameters
    ----------
    alignments_with_trace_ids : List[Dict[str, Any]]
        A list where each element contains a trace ID and its corresponding alignment.

    Returns
    -------
    List[Dict[str, Any]]
        A list where each element includes a trace ID and its move counts.
    """
    trace_encodings = []

    for item in alignments_with_trace_ids:
        trace_id = item['trace_id']
        alignment = item['alignment']

        # Initialize move count for the trace
        move_count = {}
        for log_activity, model_activity in alignment['alignment']:
            if log_activity != ">>" and log_activity not in move_count:
                move_count[log_activity] = {'log_moves': 0, 'model_moves': 0}
            elif model_activity != ">>" and model_activity not in move_count:
                move_count[model_activity] = {'log_moves': 0, 'model_moves': 0}

            if log_activity == ">>":
                move_count[model_activity]['model_moves'] += 1
            elif model_activity == ">>":
                move_count[log_activity]['log_moves'] += 1

        trace_encodings.append({'trace_id': trace_id, 'move_count': move_count})

    return trace_encodings


In [246]:
trace_encode = generate_trace_encoding(alignments_with_trace_ids)

In [163]:
def make_dataframe_for_decision_tree_with_throughput(log_df, trace_encodings, save_path):
    """
    Create a DataFrame for decision tree analysis from trace encodings and throughput times.

    Parameters
    ----------
    log_df : pd.DataFrame
        The synthetic event log as a DataFrame.
    trace_encodings : List[Dict[str, Any]]
        A list of trace encodings with their IDs.
    save_path : str
        The path to save the resulting CSV file.

    Returns
    -------
    None
    """
    # Calculate throughput time for each trace
    throughput_times = log_df.groupby('case:concept:name').agg(
        first_event=('time:timestamp', 'min'),
        last_event=('time:timestamp', 'max')
    )
    throughput_times['case:throughput_time'] = throughput_times['last_event'] - throughput_times['first_event']
    throughput_times = throughput_times[['case:throughput_time']].reset_index()

    # Flatten trace encodings and include trace IDs
    flattened_traces = []
    for item in trace_encodings:
        trace_id = item['trace_id']
        move_count = item['move_count']

        flat_trace = {'trace_id': trace_id}
        for activity, moves in move_count.items():
            if activity is not None:
                activity = activity.replace(' ', '_')
            flat_trace[f'{activity}_log_moves'] = moves['log_moves']
            flat_trace[f'{activity}_model_moves'] = moves['model_moves']
        flattened_traces.append(flat_trace)

    df_flattened = pd.DataFrame(flattened_traces)

    # Merge with throughput time
    df = pd.merge(df_flattened, throughput_times, left_on='trace_id', right_on='case:concept:name')
    df = df.drop('case:concept:name', axis=1)

    # Save to CSV
    df.to_csv(save_path, index=False)


In [247]:
# Define the save path for the resulting CSV
save_path = "/Users/Nufail/Desktop/Process_Mining/Process_Mining/conformance_checking/tests/decision_tree_data.csv"

make_dataframe_for_decision_tree_with_throughput(
    log_df=synthetic_log_deviat_test, 
    trace_encodings=trace_encode, 
    save_path=save_path
)


In [248]:
# Step 1: Load CSV Data
df_synth_tree = pd.read_csv('/Users/Nufail/Desktop/Process_Mining/Process_Mining/conformance_checking/tests/decision_tree_data.csv')

# testing

In [262]:
class DecisionTreeModel:
    def __init__(self, json_file_path='param_grid.json', random_state=42, test_size=0.2):
        self.json_file_path = json_file_path
        self.random_state = random_state
        self.test_size = test_size
        self.grid_search = None

    def load_param_grid(self):
        with open(self.json_file_path, 'r') as file:
            return json.load(file)

    def preprocess_data(self, df):
        """
        Preprocesses the input DataFrame:
        - Replaces NaN (missing activities) with a very small positive value.
        - Distinguishes between missing activities and synchronous moves.
        """
        # Fill NaN with a very small positive value to indicate missing activities
        df_filled = df.fillna(0.00001)

        # Separate features (X) and target (y)
        columns_to_exclude = ["trace_id", "None_log_moves", "None_model_moves"]
        X = df_filled.drop(columns=columns_to_exclude + ["case:throughput_time"])
        y = pd.to_timedelta(df_filled["case:throughput_time"]).apply(lambda x: x.total_seconds() / 3600)

        return X, y

    def train_model(self, df):
        """
        Trains the decision tree while respecting the distinction between synchronous moves,
        deviations, and missing activities.
        """
        X, y = self.preprocess_data(df)

        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.random_state
        )

        # Load parameter grid and initialize decision tree
        param_grid = self.load_param_grid()
        tree_regressor = DecisionTreeRegressor(random_state=self.random_state)

        # Perform Grid Search with Cross-Validation
        self.grid_search = GridSearchCV(
            estimator=tree_regressor,
            param_grid=param_grid,
            cv=5,
            scoring='neg_mean_absolute_error',
            n_jobs=-1,
            verbose=2
        )
        self.grid_search.fit(X_train, y_train)

        # Output best parameters
        print("Best Parameters found in Grid Search:", self.grid_search.best_params_)


In [263]:
# Step 1: Instantiate the model
model = DecisionTreeModel(json_file_path='param_grid.json', random_state=42, test_size=0.2)

# Step 2: Preprocess the data
X, y = model.preprocess_data(df_synth_tree)

# Step 3: Train the model
model.train_model(df_synth_tree)

# Step 4: Evaluate the model
best_model = model.grid_search.best_estimator_
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=model.test_size, random_state=model.random_state
)
y_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

# Step 5: Save the predictions
predictions_df = pd.DataFrame({
    "Actual": y_test,
    "Predicted": y_pred
})

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=60; total time=   0.0s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=60; total time=   0.0s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=60; total time=   0.0s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=60; total time=   0.0s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=60; total time=   0.0s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=80; total time=   0.0s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=80; total time=   0.0s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=80; total time=   0.0s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=80; total time=   0.0s
[CV] END max_depth=3, max_features=5, min_samples_leaf=40, min_samples_split=80; total time=   0.0s


In [265]:
# Step 1: Access the best model
best_model = model.grid_search.best_estimator_

# Step 2: Extract feature names from X
feature_names = X.columns

# Step 3: Plot the decision tree
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure(figsize=(25, 15))  # Adjust size for clarity
plot_tree(
    best_model,
    feature_names=feature_names,  # Provide the column names of X
    filled=True,                  # Fill colors for nodes based on predictions
    rounded=True,                 # Rounded corners for better visuals
    fontsize=10                   # Font size for clarity
)
plt.title("Best Decision Tree Visualization")
plt.show()

In [267]:
from sklearn.metrics import classification_report

# Assuming y_true are the true labels and y_pred are the predicted labels
report = classification_report(y_test, y_pred)

# Print the full report
print(report)

ValueError: continuous is not supported